## Discrepancy Between Residualization and Direct Fitting in QR
### Context and Problem Statement    
- **Quantile Regression (QR)** is often used to study the relationship between variables for different quantiles of the dependent variable's distribution.

- When applying QR, we have two possible approaches in the `quantile_twas` pipeline:   
1) **Residualization Approach**: `Residualize` both `x` and `y` (the main predictor and the response) against covariates `z` using `lm()`, then fit QR. We often use this as input to speed up.

2) **Direct Fitting Approach**: Directly fit QR using `x`, `y`, and `z` (covariates) without residualizing.

- Discrepancy:    
-- `Coefficients`and `p-values` differ when residualizing x and y and then applying QR, compared to applying QR directly with z in the model.    
-- This discrepancy is observed particularly when using the `QR_screen` function to compute `p-values` and the `rq()` function for `coefficient` estimation.

### Goal and Objective  
- The primary goal for this issue is to **identify the cause of the discrepancy**: 
  - Is it a problem with the residualization method, or does the difference stem from how the QR functions handle the data internally?
  
- The secondary goal is to optimize the QR process by **using residualization when possible**, as it reduces computational time. This would be particularly valuable for large datasets where direct fitting is costly.

### Potential Causes for Discrepancy    
1. **Residualization in QR**:  
   - The residuals obtained from `lm()` may not fully capture the relationship between x, y, and z in a way that is equivalent to how **QR** handles covariates internally.
   - The `lm()` method is used for ordinary least squares regression and produces residuals based on minimizing the sum of squared errors. However, QR minimizes the sum of absolute errors (L1 norm), which leads to different residual structures.

2. **QR's Internal Handling of Covariates**:
   - In QR, the handling of covariates might involve more sophisticated methods than simple residualization. For instance, QR could be adjusting the coefficients in a way that accounts for the **quantile-specific nature** of the regression, which linear models (lm) do not consider.
   - QR internally fits a **Barrodale-Roberts algorithm** (as in `rq.fit.br()`) that may handle covariates and residuals differently compared to `lm()`, and may even include special handling of dual variables and constraints not present in ordinary least squares.

3. **Possible QR-Specific Residualization**:
   - If QR applies some kind of **dual variable estimation** or other **L1-penalized adjustment**, it might explain why the direct fitting results differ from those using residuals obtained through linear regression (`lm()`).
   - **Whether QR code just uses LM for residualization?** It's possible that QR handles residuals differently, leading to the observed discrepancies.



### Residual Calculation method
This method calculates residuals for both predictor (X) and response (Y) variables by adjusting for covariates (covar) using linear regression. The residualization process removes the effect of the covariates on X and Y, isolating the variation in X and Y that is independent of the covariates.

In [6]:
library(tidyverse)
residualize <- function(X, Y, covar) {
  # Calculate residuals for X and Y with respect to covariates without scaling
  X_resid <- .lm.fit(x = cbind(1, covar), y = X)$residuals %>% as.matrix()
  Y_resid <- .lm.fit(x = cbind(1, covar), y = Y)$residuals %>% as.matrix()

  return(list(X_resid = X_resid, Y_resid = Y_resid))
}

### Steps to Reproduce
#### **Function 1**: `rq()` to get coeficient and se.
The issue can be reproduced using the following example with the `mtcars` dataset:

In [5]:
library(quantreg)
data("mtcars")

y <- as.matrix(mtcars$mpg)  # Response variable
x <- as.matrix(mtcars$wt)   # Main predictor
covariates_df <- as.matrix(mtcars[, c("hp", "qsec")])  # Covariates (hp and qsec)

# Step 1: Residualize y and x with respect to covariates (z)
residuals_result <- residualize(x, y, covariates_df)
xx_residual <- residuals_result$X_resid
yy_residual <- residuals_result$Y_resid

# Step 2: Perform quantile regression on residuals, tau = 0.9: coef = -3.45685
# NOTE: when fit the residualized x and y without intercept, in any quantiles, coef are the same.
rq_residual_model <- rq(yy_residual ~ xx_residual - 1, tau = 0.9)  # No intercept
print("Quantile regression on residuals (tau = 0.9):")
summary(rq_residual_model)

# Step 3: Perform quantile regression on residuals with intercept, tau = 0.9: coef = -6.39528
rq_residual_model_intercept <- rq(yy_residual ~ xx_residual, tau = 0.9)  # With intercept
print("Quantile regression on residuals with intercept (tau = 0.9):")
summary(rq_residual_model_intercept)

# Step 4: Perform quantile regression on original data with covariates: coef = -4.59014
rq_original_model <- rq(mpg ~ wt + hp + qsec, data = mtcars, tau = 0.9)
print("Quantile regression on original data with covariates (tau = 0.9):")
summary(rq_original_model)


[1] "Quantile regression on residuals (tau = 0.9):"



Call: rq(formula = yy_residual ~ xx_residual - 1, tau = 0.9)

tau: [1] 0.9

Coefficients:
xx_residual 
   -3.45685 

[1] "Quantile regression on residuals with intercept (tau = 0.9):"


Warning message in rq.fit.br(x, y, tau = tau, ci = TRUE, ...):
"Solution may be nonunique"



Call: rq(formula = yy_residual ~ xx_residual, tau = 0.9)

tau: [1] 0.9

Coefficients:
            coefficients lower bd upper bd
(Intercept)  3.67489      1.54171  5.55825
xx_residual -6.39528     -6.53862 -2.88415

[1] "Quantile regression on original data with covariates (tau = 0.9):"



Call: rq(formula = mpg ~ wt + hp + qsec, tau = 0.9, data = mtcars)

tau: [1] 0.9

Coefficients:
            coefficients lower bd  upper bd 
(Intercept)  13.73555    -27.65675  66.32787
wt           -4.59014     -8.77379  -3.95215
hp           -0.01156     -0.03285   0.12811
qsec          1.47432     -0.05344   3.93562

##### Observed Behavior:
The coefficients for x and their standard errors differ between the two functions.    
Residualization Approach (`rq_residual_model`) leads to different coefficients compared to the Direct Fitting Approach (`rq_original_model`).

#### **Function 2**: `QR_screen()` to get pvalue and qvalue to filter significant snps.    
In this function, we compute quantile-specific p-values and cauchy pvalue for direct X, Y, Z, or a predictor (x) by residualizing it against covariates (z) and then using quantile regression:    

In [6]:
# function QR_screen
QR_screen <- function(X, Y, Z = NULL, tau.list, threshold = 0.05, method = 'qvalue', top_count = 10, top_percent = 15) {
    p = ncol(X)
    pvec = rep(NA, p)
    ltau = length(tau.list)
    # Initialize quantile.pvalue with appropriate column names
    quantile.pvalue = matrix(NA, nrow = p, ncol = ltau, dimnames = list(colnames(X), paste("p_qr", tau.list, sep = "_")))
    y = as.matrix(Y)
    
    # If Z is provided, use it; otherwise just use the intercept
    if (!is.null(Z)) {
        zz = cbind(rep(1, nrow(y)), Z)  # Intercept and covariates
    } else {
        zz = matrix(1, nrow = nrow(y), ncol = 1)  # Only intercept
    }

    # Calculate ranks for each tau
    ranks_list = lapply(tau.list, function(tau) {
        suppressWarnings(rq.fit.br(zz, y, tau = tau)$dual - (1 - tau))
    })
    
    for (ip in 1:p) {
        x = as.matrix(X[, ip])
        VN = matrix(0, nrow = ltau, ncol = ltau)
        for (i in 1:ltau) {
            for (j in 1:ltau) {
                VN[i, j] = min(tau.list[i], tau.list[j]) - tau.list[i] * tau.list[j]
            }
        }

        # If Z is provided, compute the residuals; otherwise use x directly
        if (!is.null(Z)) {
            xstar = lm(x ~ zz - 1)$residual  # Residual after adjusting for Z
        } else {
            xstar = x  # No residual, use x directly
        }

        SN = NULL
        for (itau in 1:ltau) {
            Sn = as.matrix(t(xstar) %*% ranks_list[[itau]])
            SN = c(SN, Sn)
        }
        VN2 = matrix(outer(VN, t(xstar) %*% xstar, "*"), nrow = ltau)
        pvalue1 = pchisq(SN^2 / diag(VN2), 1, lower.tail = F)
        names(pvalue1) = tau.list
        # Store quantile-specific p-values directly in the correct place
        quantile.pvalue[ip, ] <- pvalue1
        e = solve(chol(VN2))
        SN2 = t(e) %*% SN
        # composite p-value
        pvalue = pchisq(sum(SN2^2), ltau, lower.tail = F)
    }

    # cauchy combined p-value
    pvec <- apply(quantile.pvalue, 1, Get.cauchy)
    # adjusted pvalue
    if (method == 'fdr') {
        adjusted_pvalues = p.adjust(pvec)
        method_col_name = "fdr_p_qr"
        method_quantile_names = paste0("fdr_p_qr_", tau.list)
        quantile_adjusted_pvalues = apply(quantile.pvalue, 2, p.adjust)
    } else if (method == 'qvalue') {
        adjusted_pvalues = compute_qvalues(pvec)
        method_col_name = "qvalue_qr"
        method_quantile_names = paste0("qvalue_qr_", tau.list)
        quantile_adjusted_pvalues = apply(quantile.pvalue, 2, compute_qvalues)
    } else {
        stop("Invalid method. Choose 'fdr' or 'qvalue'.")
    }

    sig_SNP_threshold = which(adjusted_pvalues < threshold)
    sig_SNP_top_count = order(adjusted_pvalues)[1:top_count]
    sig_SNP_top_percent = order(adjusted_pvalues)[1:max(1, round(length(adjusted_pvalues) * top_percent / 100))]

    sig.SNPs_names = colnames(X)[sig_SNP_threshold]
    sig.SNPs_names_top_count = colnames(X)[sig_SNP_top_count]
    sig.SNPs_names_top_percent = colnames(X)[sig_SNP_top_percent]
    phenotype_id = colnames(y)[1]

    df_result = data.frame(
        phenotype_id = phenotype_id,
        variant_id = rownames(quantile.pvalue),   # Keep all variants, not just significant ones
        p_qr = pvec,
        quantile.pvalue,        
        method_pvalue = adjusted_pvalues,
        quantile_adjusted_pvalues
    )%>%
        # Split the 'variant_id' and place 'chr', 'pos', 'ref', 'alt' at the front
        separate(variant_id, into = c("chr", "pos", "ref", "alt"), sep = "[:_]", remove = FALSE) %>%
        mutate(chr = as.numeric(gsub("chr", "", chr)), pos = as.numeric(pos)) %>%
        select(chr, pos, ref, alt, everything())

    # Rename the 'method_pvalue' column to the appropriate name based on method
    colnames(df_result)[ncol(df_result) - ltau] <- method_col_name    
    colnames(df_result)[(ncol(df_result) - ltau + 1):ncol(df_result)] <- method_quantile_names   

    if (length(sig_SNP_threshold) > 0) {
        sig_df_result = data.frame(
            phenotype_id = phenotype_id,
            variant_id = rownames(quantile.pvalue)[sig_SNP_threshold],
            p_qr = pvec[sig_SNP_threshold],
            quantile.pvalue[sig_SNP_threshold, ],        
            method_pvalue = adjusted_pvalues[sig_SNP_threshold],
            quantile_adjusted_pvalues[sig_SNP_threshold, ]
        )%>%
            # Split the 'variant_id' and place 'chr', 'pos', 'ref', 'alt' at the front
            separate(variant_id, into = c("chr", "pos", "ref", "alt"), sep = "[:_]", remove = FALSE) %>%
            mutate(chr = as.numeric(gsub("chr", "", chr)), pos = as.numeric(pos)) %>%
            select(chr, pos, ref, alt, everything())

        # Rename the 'method_pvalue' column to the appropriate name based on method
        colnames(sig_df_result)[ncol(sig_df_result) - ltau] <- method_col_name    
        colnames(sig_df_result)[(ncol(sig_df_result) - ltau + 1):ncol(sig_df_result)] <- method_quantile_names   
    } else {
        sig_df_result = data.frame()  # Return empty data frame if no significant SNPs
    }

    return(list(df_result = df_result,
                sig_p_df = sig_df_result,
                tau_list = tau.list, 
                quantile_pvalue = quantile.pvalue, 
                pvec = pvec, 
                adjusted_pvalues = adjusted_pvalues, 
                sig_SNP_threshold = sig_SNP_threshold, 
                sig.SNPs_names = sig.SNPs_names,
                sig_SNP_top_count = sig_SNP_top_count, 
                sig_SNP_top_percent = sig_SNP_top_percent))
}


### Observed Behavior: Discrepancy in SNP Selection with and without Covariates

#### Example Code:
```r
# Run QR_screen with covariates
p.screen = QR_screen(X = genotype_df, Y = phenotype_df, Z = covariates_df, tau.list, threshold = 0.05, method = "qvalue", top_count = 10, top_percent = 15) 

# Run QR_screen with residualized X and Y (no covariates)
p.screen.residual = QR_screen(X = X_residual, Y = Y_residual, Z = NULL, tau.list, threshold = 0.05, method = "qvalue", top_count = 10, top_percent = 15) 
```
#### Example Results:
[1] "There are 224 SNPs selected out of 26044 SNPs (with covariates)."     
[1] "There are 136 SNPs selected out of 26044 SNPs (residual)."

